In [3]:
import tensorflow as tf

In [18]:
import csv
import pickle
import pandas as pd
import numpy as np

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

import transformers
from transformers import BertTokenizer, BertConfig
from transformers import get_linear_schedule_with_warmup
from transformers import BertForTokenClassification, AdamW

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [19]:
from google.cloud import storage

In [20]:
torch.__version__

'2.3.0+cu121'

In [21]:
transformers.__version__

'4.40.1'

In [22]:
document_data = pd.read_csv("court_data/court_data_2022/documents.csv", sep="	") #.fillna(method="ffill")
document_data.head(10)

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
0,102383845,503,5,2,40467.0,219/14855/21,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Медінцева Н. М.,NaN,0,2022-01-02 00:00:00+02
1,102391042,4856,5,4,40105.0,560/6776/21,2021-12-29 00:00:00+02,2022-01-02 00:00:00+02,Матохнюк Д.Б.,http://od.reyestr.court.gov.ua/files/51/0c9097...,1,2022-01-04 00:00:00+02
2,102390645,4824,5,1,NaN,824/245/21,2021-12-22 00:00:00+02,2022-01-02 00:00:00+02,Нежура Вадим Анатолійович,http://od.reyestr.court.gov.ua/files/51/a53df6...,1,2022-01-04 00:00:00+02
3,102391822,2670,5,4,40114.0,640/4558/19,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/904bf7...,1,2022-01-04 00:00:00+02
4,102391796,2670,3,4,40110.0,640/10043/21,2021-12-07 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/4c9376...,1,2022-01-04 00:00:00+02
5,102391789,2670,3,4,40110.0,640/18672/21,2021-12-01 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/f1fc47...,1,2022-01-04 00:00:00+02
6,102391769,2670,3,4,40088.0,640/3619/19,2021-11-10 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/98d4ee...,1,2022-01-04 00:00:00+02
7,102391764,2670,3,4,40114.0,640/4230/19,2021-11-05 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/dbaa6e...,1,2022-01-04 00:00:00+02
8,102391757,2670,3,4,40077.0,640/1726/20,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/145f06...,1,2022-01-04 00:00:00+02
9,102388442,1570,3,4,40074.0,420/20761/21,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Потоцька Н.В.,http://od.reyestr.court.gov.ua/files/51/5514da...,1,2022-01-04 00:00:00+02


In [23]:
document_data.shape

(5822649, 12)

In [24]:
import requests

In [25]:
document_data["doc_url"].to_numpy()

array([nan,
       'http://od.reyestr.court.gov.ua/files/51/0c9097fb28ccabfcd4db8d140fd667b7.rtf',
       'http://od.reyestr.court.gov.ua/files/51/a53df6d681ad7a5921e67e15a6bce992.rtf',
       ..., nan, nan, nan], dtype=object)

In [26]:
document_data["doc_url"][1]

'http://od.reyestr.court.gov.ua/files/51/0c9097fb28ccabfcd4db8d140fd667b7.rtf'

In [27]:
requests.get(document_data["doc_url"][1])

<Response [200]>

### Uploading data to Cloud Storage

In [28]:
SUBFOLDER = "data_50_rtf/"

In [29]:
storage_client = storage.Client()
bucket = storage_client.bucket("court_data_bert")

In [30]:
def download_court_file(url: str, doc_id: str):
    rtf_court_raw_filename = url.split("/")[-1]
    result_filename = SUBFOLDER + str(doc_id) + "_" + rtf_court_raw_filename
    blob = bucket.blob(result_filename)
    
    response = requests.get(url)
    response.raise_for_status()
    
    blob.upload_from_string(response.content, content_type=response.headers.get('content-type'))

#### Uploading 50 instances

In [31]:
document_data

,doc_id,court_code,judgment_code,justice_kind,category_code,cause_num,adjudication_date,receipt_date,judge,doc_url,status,date_publ
0,102383845,503,5,2,40467.0,219/14855/21,2021-12-30 00:00:00+02,2022-01-02 00:00:00+02,Медінцева Н. М.,NaN,0,2022-01-02 00:00:00+02
1,102391042,4856,5,4,40105.0,560/6776/21,2021-12-29 00:00:00+02,2022-01-02 00:00:00+02,Матохнюк Д.Б.,http://od.reyestr.court.gov.ua/files/51/0c9097...,1,2022-01-04 00:00:00+02
2,102390645,4824,5,1,NaN,824/245/21,2021-12-22 00:00:00+02,2022-01-02 00:00:00+02,Нежура Вадим Анатолійович,http://od.reyestr.court.gov.ua/files/51/a53df6...,1,2022-01-04 00:00:00+02
3,102391822,2670,5,4,40114.0,640/4558/19,2021-11-02 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/904bf7...,1,2022-01-04 00:00:00+02
4,102391796,2670,3,4,40110.0,640/10043/21,2021-12-07 00:00:00+02,2022-01-02 00:00:00+02,Клименчук Н.М.,http://od.reyestr.court.gov.ua/files/51/4c9376...,1,2022-01-04 00:00:00+02
...,...,...,...,...,...,...,...,...,...,...,...,...
5822644,108198825,444,5,2,40469.0,216/1387/22,2022-12-26 00:00:00+02,2022-12-31 00:00:00+02,КУЗНЕЦОВ Р. О.,NaN,0,2024-04-22 00:00:00+03
5822645,108205032,1304,5,2,40469.0,461/7106/22,2022-12-29 00:00:00+02,2022-12-31 00:00:00+02,Стрельбицький В. В.,NaN,0,2024-04-22 00:00:00+03
5822646,108199852,2609,5,2,40469.0,760/18063/22,2022-12-08 00:00:00+02,2022-12-31 00:00:00+02,Криворот О. О.,NaN,0,2024-04-22 00:00:00+03
5822647,108199868,2609,5,2,40469.0,760/17205/22,2022-12-12 00:00:00+02,2022-12-31 00:00:00+02,Криворот О. О.,NaN,0,2024-04-22 00:00:00+03


In [32]:
for index, row in document_data.notna()[:50].iterrows():
    print("Processing -> ", document_data["doc_url"][index])

    # if document_data["doc_url"][index].isna():
    #     continue
        
    if pd.isna(document_data.at[index, "doc_url"]):
        continue
        
    download_court_file(
        document_data["doc_url"][index], 
        document_data["doc_id"][index],
    )

Processing ->  nan
Processing ->  http://od.reyestr.court.gov.ua/files/51/0c9097fb28ccabfcd4db8d140fd667b7.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/a53df6d681ad7a5921e67e15a6bce992.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/904bf72496663177e3539502ce03f0d5.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/4c9376af59893fd2957aa68f39b7d654.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/f1fc470bf95c0e99c9e33cfa46d3ec71.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/98d4eef7fb8b9655635ffbf662f0aa72.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/dbaa6e37b1bbd685e1926eadfc788ba5.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/145f06c30b717a6706a9080aad6e0c48.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/5514da0c566c412ff53cdf09edc51f80.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/654492e8676f335c07fef5e50597108f.rtf
Processing ->  http://od.reyestr.court.gov.ua/files/51/e69098